In [4]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import os
os.chdir('..')

cdm = importr('CDM')
cdm = importr('ltm')

In [2]:
def pythonify(r_matrix):
    robjects.r('M <- as.matrix(%s)' % r_matrix)
    n, m = robjects.r.dim(robjects.r.M)
    print('%s has size %s x %s' % (r_matrix, n, m))
    return [[robjects.r.M.rx(1 + i, 1 + j)[0] for j in range(m)] for i in range(n)]

In [35]:
t = pythonify('Science[c(1, 2)]')

Science[c(1, 2)] has size 392 x 2


In [36]:
import pandas as pd

triplets = []
for i, line in enumerate(t):
    for j, elt in enumerate(line):
        triplets.append((i, j, elt))
df = pd.DataFrame(triplets, columns=('i', 'j', 'v'))

In [37]:
df.head()

,i,j,v
0,0,0,strongly agree
1,0,1,strongly agree
2,1,0,agree
3,1,1,strongly agree
4,2,0,agree


In [45]:
ordered_classes = list(df['v'].unique()[::-1])

In [46]:
ordered_classes

['strongly disagree', 'disagree', 'agree', 'strongly agree']

In [39]:
df['val'] = df['v'].map(ordered_classes.index)

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
train, test = train_test_split(df)

In [42]:
len(train), len(test)

(588, 196)

In [43]:
df['i'].max(), df['j'].max()

(391, 1)

In [44]:
train[['i', 'j', 'val']].to_csv('/tmp/train.csv', header=False, index=False)
test[['i', 'j', 'val']].to_csv('/tmp/test.csv', header=False, index=False)

In [4]:
import json
from qmatrix import QMatrix

def create_data(dataset_name, data, qmatrix):
    py_data = pythonify(data)
    py_qmatrix = pythonify(qmatrix)
    with open('data/%s.json' % dataset_name, 'w') as f:
        f.write(json.dumps({'student_data': py_data}))
    robjects.r('mod <- din(%s, q.matrix=%s, rule="DINA")' % (data, qmatrix))
    
    nb_components = len(py_qmatrix[0])
    guess = list(robjects.r('mod$guess$est'))
    slip = list(robjects.r('mod$slip$est'))
    q = QMatrix(nb_components, py_qmatrix, slip, guess)
    q.display_qmatrix()
    q.save('qmatrix-%s' % dataset_name)

In [5]:
create_data('ecpe', 'data.ecpe$data[,2:29]', 'data.ecpe$q.matrix')

data.ecpe$data[,2:29] has size 2922 x 28
data.ecpe$q.matrix has size 28 x 3
---------------------------------------------------------------------------------
DINA MODEL 
** 2016-08-30 14:44:01 
---------------------------------------------------------------------------------
Iter.  1  :  14:44:02 ,   loglike =  -48100.12  / max. param. ch. :  0.536317  / relative deviance change :  1
Iter.  2  :  14:44:02 ,   loglike =  -43378.32  / max. param. ch. :  0.068317  / relative deviance change :  0.108852
Iter.  3  :  14:44:02 ,   loglike =  -43202.24  / max. param. ch. :  0.047803  / relative deviance change :  0.004076
Iter.  4  :  14:44:02 ,   loglike =  -43094.75  / max. param. ch. :  0.037809  / relative deviance change :  0.002494
Iter.  5  :  14:44:02 ,   loglike =  -43022.48  / max. param. ch. :  0.028244  / relative deviance change :  0.00168
Iter.  6  :  14:44:02 ,   loglike =  -42972.78  / max. param. ch. :  0.021244  / relative deviance change :  0.001156
Iter.  7  :  14:44:02 , 

In [8]:
create_data('timss2003', 'data.timss03.G8.su$data[,3:25]', 'data.timss03.G8.su$q.matrix')

data.timss03.G8.su$data[,3:25] has size 757 x 23
data.timss03.G8.su$q.matrix has size 23 x 13
---------------------------------------------------------------------------------
DINA MODEL 
** 2016-08-24 20:20:01 
---------------------------------------------------------------------------------
Iter.  1  :  20:20:06 ,   loglike =  -11732.66  / max. param. ch. :  0.529739  / relative deviance change :  1
Iter.  2  :  20:20:10 ,   loglike =  -9893.177  / max. param. ch. :  0.056788  / relative deviance change :  0.185934
Iter.  3  :  20:20:13 ,   loglike =  -9795.977  / max. param. ch. :  0.059353  / relative deviance change :  0.009922
Iter.  4  :  20:20:17 ,   loglike =  -9746.685  / max. param. ch. :  0.03596  / relative deviance change :  0.005057
Iter.  5  :  20:20:21 ,   loglike =  -9719.582  / max. param. ch. :  0.017775  / relative deviance change :  0.002789
Iter.  6  :  20:20:25 ,   loglike =  -9702.399  / max. param. ch. :  0.013763  / relative deviance change :  0.001771
Iter. 

In [9]:
create_data('timss2007', 'data.timss07.G4.lee$data[,4:28]', 'data.timss07.G4.lee$q.matrix')

data.timss07.G4.lee$data[,4:28] has size 698 x 25
data.timss07.G4.lee$q.matrix has size 25 x 15
---------------------------------------------------------------------------------
DINA MODEL 
** 2016-08-24 20:41:27 
---------------------------------------------------------------------------------
Iter.  1  :  20:41:41 ,   loglike =  -7439.882  / max. param. ch. :  0.673591  / relative deviance change :  1
Iter.  2  :  20:41:53 ,   loglike =  -6581.138  / max. param. ch. :  0.06332  / relative deviance change :  0.130486
Iter.  3  :  20:42:07 ,   loglike =  -6527.267  / max. param. ch. :  0.041671  / relative deviance change :  0.008253
Iter.  4  :  20:42:20 ,   loglike =  -6494.444  / max. param. ch. :  0.045942  / relative deviance change :  0.005054
Iter.  5  :  20:42:32 ,   loglike =  -6470.099  / max. param. ch. :  0.04176  / relative deviance change :  0.003763
Iter.  6  :  20:42:43 ,   loglike =  -6451.249  / max. param. ch. :  0.031021  / relative deviance change :  0.002922
Iter.

In [9]:
q_test = [[1, 1, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]]
nb_items = len(q_test)
nb_components = len(q_test[0])
q = QMatrix(nb_components, q_test, slip=[0] * nb_items, guess=[0] * nb_items)
q.display_qmatrix()
q.save('qmatrix-%s' % 'tmp')

prefix is now tmp0
prefix is now tmp1
prefix is now tmp2
prefix is now tmp0
11. 0 0
1.. 0 0
.1. 0 0
..1 0 0


In [3]:
q = QMatrix()
q.load('qmatrix-tmp')
q.init_test(set())
print(q.p_test)
# q.next_item([], [])
q.estimate_parameters([0], [1])

prefix is now tmp0
prefix is now tmp1
prefix is now tmp2
prefix is now tmp0
[0.5, 0.5, 0.5]


ZeroDivisionError: float division by zero